In [16]:
import pandas as pd
import numpy as np

In [17]:
trip = pd.read_csv('data/tripP')
train = pd.read_csv('data/trip_train.csv', usecols=['id'])
test = pd.read_csv('data/trip_test.csv', usecols=['id'])

In [18]:
train = train.merge(trip, on='id', how='left')
test = test.merge(trip, on='id', how='left')

In [19]:
train.head(1)

,id,duration,date,start_station_id,subscription_type,time,year,month,day,dayofweek,date.1,holiday
0,907649,396,2015-08-27,50,1,516,2015,8,27,4,NaN,NaN


In [20]:
from sklearn import preprocessing
cols = ['start_station_id', 'subscription_type', 'time', 'year', 'month', 'day','dayofweek']
X = train[cols].apply(lambda x: preprocessing.MinMaxScaler().fit_transform(x), axis=0)
y = train.duration

/home/champaine/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/champaine/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/champaine/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1,

In [21]:
X_test = test[cols].apply(lambda x: preprocessing.MinMaxScaler().fit_transform(x), axis=0)
y_test = test.duration

/home/champaine/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/champaine/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/champaine/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1,

In [22]:
X.sample(5)

,start_station_id,subscription_type,time,year,month,day,dayofweek
399552,0.792683,0.0,0.517026,0.5,0.181818,0.666667,0.666667
289424,0.524390,1.0,0.599722,0.5,0.818182,0.700000,0.333333
544119,0.451220,1.0,0.543433,0.5,0.545455,0.733333,0.333333
478245,0.768293,1.0,0.703961,1.0,0.272727,0.200000,0.166667
482452,0.597561,1.0,0.617095,0.5,0.727273,0.600000,0.666667


In [23]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import FeatureAgglomeration
clustering = KMeans(n_clusters=11,n_jobs=-1)
#clustering = DBSCAN(n_jobs=-1)
#clustering = FeatureAgglomeration(n_clusters=7)

In [24]:
clustering.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=11, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [25]:
X['cluster'] = clustering.predict(X)

In [26]:
X_test['cluster'] = clustering.predict(X_test)

In [28]:
X.sample(3)

,start_station_id,subscription_type,time,year,month,day,dayofweek,cluster
400850,0.914634,1.0,0.768589,0.5,0.181818,0.100000,0.166667,9
308380,0.878049,1.0,0.341209,1.0,0.090909,0.033333,0.000000,5
161847,0.707317,1.0,0.922168,0.5,0.454545,0.600000,0.500000,6


In [29]:
X_test.sample(3)

,start_station_id,subscription_type,time,year,month,day,dayofweek,cluster
94268,0.524390,1.0,0.462126,0.5,0.636364,0.733333,0.833333,6
85592,0.707317,1.0,0.763725,0.5,0.454545,0.333333,0.333333,9
100640,0.719512,1.0,0.537874,0.5,0.000000,0.800000,0.833333,6


In [ ]:
from sklearn import neural_network
pred = neural_network.MLPRegressor().fit(X, y).predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(pred, y_test)

In [ ]:
predictionDF = pd.DataFrame(pred,columns={"duration"})

In [ ]:
predictionDF.loc[:,'id'] = test['id']

In [ ]:
predictionDF.to_csv(path_or_buf='data/neural_networks_with_clustering',sep=',',header=True,columns=['id','duration'],index=False)